In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

In [14]:
def conv_model_fn(features, labels, mode):
    # - 1 is to autotune bactch-size, features sunt numarul total de intrari, 
    #adica 784 * batch_size, iar in dreapta este forma
    reshaped_inputs = tf.reshape(features, [-1, 28, 28, 1])
    #input [batch_size, 28, 28, 1] ==> output[batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
                            inputs=reshaped_inputs,
                            activation=tf.nn.relu,
                            filters=32,
                            kernel_size=[5,5],
                            padding='same'
                            )
    #input [batch_size, 28, 28, 32] ==> output[batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2,2], strides=2)
    #input [batch_size, 14, 14, 32] ==> output[batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
                            inputs=pool1,
                            activation=tf.nn.relu,
                            filters=64,
                            kernel_size=[5,5],
                            padding='same' #adica sa pastreze forma inputului, de 28 * 28
                            )
    #input [batch_size, 14, 14, 64] ==> output[batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    #Dense Layer, flat ==> tensorul redevine 2d, de marime [batch_size, output_pool2]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    #fully-connected layer de 1024 de neuroni ==> output[batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    #dropout ==> 40% din date vor fi randomly dropped in training
    #se verifica daca modul este training, pentru a face sau nu dropout ==> output[batch_size, 1024]
    drop = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)
    #Logits sau estimare, presupun
    logits = tf.layers.dense(inputs=drop, units=10)
    loss = None
    train_op = None
    # # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        #onehot ==> unde este valoare 1 in tensor, care declanseaza label-ul, 10 clase (depth)
        one_hotlabels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        #loss este scalar, valoarea returnata de softmax
        loss = tf.losses.softmax_cross_entropy(onehot_labels=one_hotlabels, logits=logits)
    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="SGD")
    #Cauta in tensor pe orizontala argmax, adica unde este valoarea 1, cautam valoarea cea mai mare 
    #in dimensiunea cu index 1, apoi aplicam softmax pentru a obtine o probabilitate
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    return model_fn_lib.ModelFnOps(mode=mode, predictions=predictions, train_op=train_op, loss=loss)

In [15]:
def main(argv):
    mnist = learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    test_data = mnist.test.images
    test_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    mnist_estim = learn.Estimator(model_fn=conv_model_fn, model_dir="MNIST_data/")
    #Let's set up some logging to see the progress with logging_hook
    logs = {"probabilities" : "softmax_tensor"}
    log_hook = tf.train.LoggingTensorHook(tensors=logs, every_n_iter=50)
    mnist_estim.fit(x=train_data, y=train_labels, batch_size=100, steps=20000, monitors=[log_hook])
    #Accuracy metric for evaluation
    metric = {
        #metric_fn - functia care calculeaza si returneaza accuray, putem folosi fct default accuracy din tf.metrics
        #prediction key, cheia care contine label-urile
        "accuracy": learn.MetricSpec(metric_fn=tf.metrics.accuracy, prediction_key="classes")
    }
    #Facem evaluare
    evalul = mnist_estim.evaluate(x=test_data, y=test_labels, metrics=metric)
    print(evalul)

In [ ]:
if __name__ == "__main__":
    tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_model_dir': None, '_master': '', '_task_type': None, '_environment': 'local', '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9afcca70b8>, '_task_id': 0, '_evaluation_master': '', '_is_chief': True, '_num_worker_replicas': 0}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimato

/home/flavius/anaconda3/lib/python3.5/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into MNIST_data/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.30034
INFO:tensorflow:probabilities = [[ 0.07334762  0.09645092  0.09379227  0.1020344   0.11243463  0.10517013
   0.11542577  0.12032797  0.09507286  0.08594338]
 [ 0.08457483  0.09649383  0.10131261  0.10580406  0.10211761  0.10003387
   0.11624278  0.10834341  0.09771004  0.08736698]
 [ 0.07731293  0.09835514  0.08390491  0.11773082  0.11294403  0.10046723
   0.1118436   0.11918855  0.08111848  0.09713426]
 [ 0.07763945  0.09753578  0.0931226   0.1098754   0.12504564  0.09741591
   0.11860139  0.10550665  0.09284841  0.08240876]
 [ 0.08334383  0.10159022  0.08611266  0.11232153  0.10084321  0.11942852
   0.10577449  0.095222    0.11403917  0.08132433]
 [ 0.08501171  0.09069485  0.09474856  0.11539796  0.0932812   0.10224444
   0.11005077  0.1116351   0.10274685  0.09418861]
 [ 0.08523989  0.09918077  0.08750451  0.11731132  0.09086074  0